In [1]:
#hide
#default_exp test
from nbdev.showdoc import *
from nbdev.export import notebook2script

# Tests

> Functions and scripts for performing automated testing on a KiCad project

* toc: true

In [2]:
#export
import os
import subprocess

from fastcore.script import *
from nbdev.test import *
from nbdev.test import nbglob, num_cpus, parallel, _test_one, Path
from kicad_helpers import *
from kicad_helpers.utilities import _set_root

In [3]:
#hide
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)
print(subprocess.check_output(f"kh_update --overwrite --root { root }", shell=True).decode("utf-8"))

In [4]:
#export
@call_parse
def test_notebooks(fname:Param("A notebook name or glob to convert", str)=None,
                   flags:Param("Space separated list of flags", str)=None,
                   n_workers:Param("Number of workers to use", int)=None,
                   verbose:Param("Print errors along the way", bool_arg)=True,
                   timing:Param("Timing each notebook to see the ones are slow", bool)=False,
                   pause:Param("Pause time (in secs) between notebooks to avoid race conditions", float)=0.5,
                   root:Param("project root directory", str)="."):
    "Test in parallel the notebooks matching `fname`, passing along `flags`"
    root = _set_root(root)
    if flags is not None: flags = flags.split(' ')
    if fname is None:
        fname = os.path.join(root, "tests", "*.ipynb")
    files = nbglob(fname, recursive=False)
    files = [Path(f).absolute() for f in sorted(files)]
    assert len(files) > 0, "No files to test found."
    if n_workers is None: n_workers = 0 if len(files)==1 else min(num_cpus(), 8)
    # make sure we are inside the tests folder
    os.chdir(os.path.join(root, "tests"))
    results = parallel(_test_one, files, flags=flags, verbose=verbose, n_workers=n_workers, pause=pause)
    passed,times = [r[0] for r in results],[r[1] for r in results]
    if all(passed): print("All tests are passing!")
    else:
        msg = "The following notebooks failed:\n"
        raise Exception(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))
    if timing:
        for i,t in sorted(enumerate(times), key=lambda o:o[1], reverse=True):
            print(f"Notebook {files[i].name} took {int(t)} seconds")

Run all tests in the `tests` directory.

```sh
> kh_test
```

In [5]:
#hide_input
print(subprocess.check_output(f"kh_test --root { root }", shell=True).decode("utf-8"))

testing /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/tests/Tests.ipynb
All tests are passing!



In [6]:
#hide
notebook2script()

Converted 00_actions.ipynb.
Converted 01_test.ipynb.
Converted 02_utilities.ipynb.
Converted index.ipynb.
